In [1]:
%defaultDatasource jdbc:h2:mem:db

In [2]:
DROP TABLE IF EXISTS Drug;
DROP TABLE IF EXISTS Pathology;
DROP TABLE IF EXISTS DrugUse;

CREATE TABLE Drug (
  code INT NOT NULL,
  name VARCHAR(200) NOT NULL,
  PRIMARY KEY(code)
) AS SELECT
    code,
    name
FROM CSVREAD('data/drug.csv');

CREATE TABLE Pathology (
  code INT NOT NULL,
  name VARCHAR(200) NOT NULL,
  PRIMARY KEY(code)
) AS SELECT
    code,
    name
FROM CSVREAD('data/pathology.csv');

CREATE TABLE DrugUse (
  idPerson INT NOT NULL,
  codePathology INT NOT NULL,
  codeDrug INT NOT NULL,
  FOREIGN KEY(codePathology)
    REFERENCES Pathology(code)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
  FOREIGN KEY(codeDrug)
    REFERENCES Drug(code)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
) AS SELECT
    idPerson,
    codePathology,
    codeDrug
FROM CSVREAD('data/drug-use.csv');

CREATE TABLE SideEffect(
  idPerson INT NOT NULL,
  codePathology INT NOT NULL,
  FOREIGN KEY(codePathology)
    REFERENCES Pathology(code)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
) AS SELECT idPerson, codePathology
FROM CSVREAD('data/sideeffect.csv');

In [ ]:
SELECT * FROM Drug;
SELECT * FROM Pathology;
SELECT * FROM DrugUse;
SELECT * FROM SideEffect;

# FDA Adverse Event Reporting System (FAERS)

Considere a base de dados que reúne relatos de efeitos adversos de drogas: [FDA Adverse Event Reporting System (FAERS)](https://open.fda.gov/data/faers/).

As fontes de dados são publicadas e documentadas em: [Latest Quarterly Data Files](https://www.fda.gov/drugs/fda-adverse-event-reporting-system-faers/fda-adverse-event-reporting-system-faers-latest-quarterly-data-files)

# Interação entre medicamentos
Considere como uma interação toda vez que um medicamento A é tomado em conjunto com um medicamento B por uma pessoa. Crie uma consulta que retorne para cada par de medicamento A,B o número de interações, somente para aqueles medicamentos que são tomados em conjunto mais de 30 vezes (passaremos a chamar de **grupo mais 30**).

In [4]:
DROP VIEW IF EXISTS Interacoes;
CREATE VIEW Interacoes AS
SELECT DISTINCT DUA.codeDrug DrugA, DUB.codeDrug DrugB, COUNT(*) QtdInteracoes
        FROM DrugUse DUA, DrugUse DUB
        WHERE DUA.idPerson = DUB.idPerson AND DUA.codeDrug < DUB.codeDrug
        GROUP BY DrugA, DrugB
        HAVING QtdInteracoes > 30;

SELECT * FROM Interacoes;

## Tabela Resultante

Se estiver demorando muito para gerar a tabela, você poderá usar a seguinte tabela resultante pronta. Essa tabela tem todas as interações, você ainda precisará filtrar o **grupo mais 30**.

In [3]:
DROP TABLE IF EXISTS DrugInteraction;
CREATE TABLE DrugInteraction (
  druga INT NOT NULL,
  drugb INT NOT NULL,
  weight INT NOT NULL,
  PRIMARY KEY(druga, drugb)
) AS SELECT
    druga,
    drugb,
    weight
FROM CSVREAD('data/drug-interaction.csv');

DROP VIEW IF EXISTS Interacoes30;
CREATE VIEW Interacoes30 AS
SELECT DI.druga DrugA, DI.drugb DrugB, DI.weight QtdInteracoes
  FROM DrugInteraction DI 
  WHERE DI.weight>30;

In [5]:
SELECT * From DrugInteraction

# Suporte

### Qual o suporte que cada uma das associações tem?

Gere uma consulta que apresente o suporte do **grupo mais 30**. Ordene o resultado por ordem de suporte. 

Observação: Para que o resultado da divisão seja fracionária, converta o numerador em `DOUBLE` usando:
~~~sql
CAST(campo AS DOUBLE)
~~~

In [28]:
SELECT I.DrugA, I.DrugB, CAST(I.QtdInteracoes AS DOUBLE)/(SELECT COUNT(*) FROM DrugInteraction) Suporte
    FROM Interacoes30 I
    ORDER BY Suporte

# Confiança

### Qual a confiança que cada uma das associações tem?

Gere uma consulta que apresente a confiança do **grupo mais 30**. Ordene o resultado por ordem de confiança.

In [50]:
SELECT I.DrugA, I.DrugB, CAST(I.QtdInteracoes AS DOUBLE)/(SELECT SUM(DI.weight) FROM DrugInteraction DI WHERE DI.druga = I.DrugA) Confianca
    FROM Interacoes30 I
    ORDER BY Confianca

# Efeito Colateral

Para cada medicamento, qual o percentual do medicamento A que tem efeito colateral junto com o medicamento B, quando comparado com seu uso sem o medicamento B.

In [4]:
DROP VIEW MedicamentoEC IF EXISTS;
CREATE VIEW MedicamentoEC AS
SELECT DU.codeDrug, COUNT(*) Numero 
    FROM DrugUse DU, SideEffect SE 
    WHERE DU.idPerson = SE.idPerson AND DU.codePathology = SE.codePathology
    GROUP BY codeDrug;

SELECT * FROM MedicamentoEC

In [5]:
DROP VIEW MedicamentoECB IF EXISTS;
CREATE VIEW MedicamentoECB AS
SELECT DI.DrugA DrugA, DI.DrugB DrugB, COUNT(*) Numero
    FROM DrugUse DUA, DrugUse DUB, DrugInteraction DI, SideEffect SE
    WHERE DI.DrugA = DUA.codeDrug AND DUA.codePathology = SE.codePathology AND DUA.idPerson = SE.idPerson AND DI.DrugB = DUB.codeDrug AND DUB.idPerson = SE.idPerson AND DUB.codePathology = SE.codePathology
    GROUP BY DrugA, DrugB;

SELECT * FROM MedicamentoECB;

In [6]:
SELECT ECB.DrugA, ECB.DrugB, CAST(ECB.Numero AS DOUBLE)/EC.Numero Porcentagem
    FROM MedicamentoECB ECB, MedicamentoEC EC
    WHERE ECB.DrugA = EC.codeDrug;